**извлечение признаков из текста на естественном языке**

класстеризатор текстов

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
2425,"Coldplay, «Кино» и группировка «Ленинград» стали самыми популярными исполнителями среди петербуржцев в уходящем году, сообщила «Радио Балтика» региональный представитель компании «Яндекс» Александ...",culture
532,Пропавшая в Минске 79-летняя пенсионерка нашлась\n\nопубликовано: 6 декабря 2016 в 12:58\n\nобновлено: 7 декабря 2016 в 9:48\n\nTUT.BY\n\nФото предоставлено ПСО «Ангел»\n\nТрое суток в столице и М...,incident


## токенизация и очистка

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in ['в','на','за','для','под','над'] ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов (предлогов)
    return ' '.join( [ t.strip() for t in tt if t ] )
    

##  CountVectorizer + TF-IDF

In [8]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(preprocessor=preprocessor)
tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f7eddf11730>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [9]:
# размер словаря
len(tf.vocabulary_)

37661

## формируем датасеты

In [10]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 37661)

## кластеризируем

In [22]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [23]:
%%time

n_clusters = len(set(data['tag']))
clust = KMeans(n_clusters=n_clusters, random_state=get_seed() )
clust.fit(X)

CPU times: user 56.1 s, sys: 23.2 ms, total: 56.1 s
Wall time: 48.3 s


In [31]:
# set(clust.labels_)

In [25]:
data['cluster'] = clust.labels_

## тестируем

In [26]:
data[ data['cluster']==0 ].sample(10)

,text,tag,cluster
2259,"В Южной Корее на военной базе вблизи города Ульсан прогремел взрыв. Об этом сообщает Yonhap News\n\nПо данным агентства, в результате пострадали 20 человек. Большинство из них получили ожоги разли...",incident,0
2288,"Огонь охватил 350 квадратных метров.\n\nВечером 12 декабря 2016 года в Екатеринбурге загорелся банный комплекс ""Русские бани"", расположенный на улице Водонасосной. Огонь захватил 350 квадратных ме...",incident,0
579,"В Могилеве горело училище олимпийского резерва, эвакуировали 35 человек\n\n1 декабря 2016 в 11:06\n\nTUT.BY\n\nПожар в училище олимпийского резерва в Могилеве произошел 30 ноября около полудня. Ды...",incident,0
2193,"РИАМО - 13 дек. Пожарные потушили возгорание деревянных конструкций на площади 200 квадратных метров на складе на Мироновской улице в Восточном округе Москвы, пострадавших нет, говорится в сообщен...",incident,0
524,"В Ельске из-за пожара сразу две семьи лишились жилья\n\n7 декабря 2016 в 15:35\n\nTUT.BY\n\nПожар в Ельске, произошедший поздно вечером 6 декабря, оставил без крыши над головой сразу две семьи. По...",incident,0
581,В США более 10 человек погибли в результате лесных пожаров и торнадо\n\n1 декабря 2016 в 6:53\n\nРусская служба Би-би-си\n\nВ юго-восточных штатах США более десяти человек погибли в результате тор...,incident,0
569,В Полоцке горел архив в библиотеке. Эвакуировали более 60 человек\n\n2 декабря 2016 в 9:01\n\nTUT.BY\n\nПожар произошел в архиве центральной библиотеки имени Франциска Скорины в Полоцке вечером 1 ...,incident,0
2297,"ВЛАДИВОСТОК, 13 дек – РИА Новости. Пострадавшая при взрыве газа в Советской Гавани скончалась в больнице Хабаровска, сообщил РИА Новости представитель КГБУЗ ""Краевая Клиническая Больница №2"".\n\nП...",incident,0
2303,"07:50 сегодня 269 0\n\nОб этом РИА Новости сообщили в краевой клинической больнице №2, куда женщина поступила с сильными ожогами после инцидента.\n\nЧП произошло утром 7 декабря в городе Советская...",incident,0
2260,"На военной базе в южнокорейском городе Ульсан во вторник прогремел взрыв. База находится в 410 километрах к юго-востоку от Сеула, сообщают журналисты.\n\nОколо 20 человек получили ранения в резуль...",incident,0


In [27]:
data[ data['cluster']==1 ].sample(10)

,text,tag,cluster
1852,"Россия хочет нормализации отношений с Японией, заявил президент России Владимир Путин в интервью японской телекомпании Nippon TV (NTV).\n\n""Начиная с 1956 года, когда были восстановлены дипломатич...",politics,1
2953,Путин: У России нет территориальных проблем с Японией\r\n\r\nПерестрелка силовиков в Княжичах: африканизация Украины и киевский неофеодализм\r\n\r\n,politics,1
1875,"Токио, 13 декабря. Власти Японии во вторник обнародовали программу визита президента России Владимира Путина, который должен посетить страну 15-16 декабря. Российский лидер прибудет в префектуру Я...",politics,1
1729,"Крупнейший японский банк Sumitomo Mitsui и банк РФ ВТБ намерены подписать соглашение о сотрудничестве, передает японское агентство Kyodo.\n\nKyodo отмечает, что соглашение предусматривает, что сто...",economics,1
1846,"Министр иностранных дел России Сергей Лавров заявил, что высказывания об угрозе конфликта с РФ на Балканах странно слышать от Хорватии, но Москва готова обсудить факты Загреба при их наличии.\n\nР...",politics,1
1876,"МИД Японии официально объявил программу визита в страну президента России Владимира Путина 15—16 декабря.\n\nКак передаёт ТАСС со ссылкой на японское дипведомство, после прибытия в аэропорт префек...",politics,1
1727,Одна из крупнейших японских кредитных организаций Sumitomo Mitsui и российский банк ВТБ планируют подписать договор о сотрудничестве. Документ предусматривает двустороннее представление клиентов д...,economics,1
1849,"Отсутствие мирного договора между Россией и Японией является анахронизмом, считает президент Владимир Путин. Выдержку из его интервью телеканалу Nippon TV приводит РИА Новости.\n\n«Этот анахронизм...",politics,1
466,"Путин рассказал о желании ""успешно завершить карьеру"" и начать путешествовать\n\n6 декабря 2016 в 6:46\n\nАнастасия Юдицкая, РБК\n\nПрезидент Владимир Путин на встрече с работниками ООО «Этерно» в...",politics,1
1575,"ТОКИО, 13 декабря. /Корр. ТАСС Игорь Беляев/. Президент РФ Владимир Путин не исключает возможность ведения совместной с Японией экономической деятельности на южных Курильских островах. Такое заявл...",politics,1


In [28]:
data[ data['cluster']==2 ].sample(10)

,text,tag,cluster
18,"Путин поручил внести в Думу проект закона о 12-месячной военной службе...\nРФ доработать и внести в Госдуму законопроект, предусматривающий с 1\nянваря 2008 года переход на 12-месячную военную слу...",politics,2
3055,"РЖД потеряла интерес к <object>ТГК-14</object>, которой владела вместе с группой ЕСН, и может продать свою долю партнеру по консорциуму. Однако в конце концов контроль над компанией может перейти ...",economics,2
251,"Большой театр отверг Мстислава Ростроповича Маэстро приступил к репетициям\nс оркестром 23 ноября, но обнаружил, что в расписании их заявлено меньше,\nчем в контракте.Напомним, что великий дирижер...",culture,2
822,Смотрители Мюнхенского зоопарка опубликовали видео новорожденного белого медвежонка\r\n\r\n2 декабря 2016 в 10:37\r\n\r\nВидео: Reuters\r\n\r\nУ белой медведицы Джованны в зоопарке Мюнхена 21 нояб...,social,2
650,"Стартапы Беларуси соберутся на конференции ""СТАРТАП ГОДА 2016""\n\n6 декабря 2016 в 9:00\n\n*На правах рекламы\n\nУходящий год ознаменуется знаковым для стартап-движения событием. 17 декабря в отел...",tech,2
2780,\r\n\r\nЖалобы от обладателей устройств от компании Apple были учтены разработчиками нового обновления календаря iCloud. В результате в новом программном продукте будут учтены пожелания пользовате...,tech,2
850,Набирающая популярность в сети российская поэтесса Ах Астахова приедет с творческим вечером в Минск\n\n8 декабря 2016 в 19:46\n\nAFISHA.TUT.BY\n\n14 февраля в Минске будет разливаться искусство сл...,culture,2
834,"Киркоров прокомментировал конфликт с Маруани: ""Все сделали пранкеры""\n\n30 ноября 2016 в 15:20\n\nLenta.ru\n\nПевец Филипп Киркоров считает обвинения лидера группы Space Дидье Маруани в плагиате а...",social,2
2792,"17:27 - 12.12.2016\n\nАлексей Жаров, глава Роскомнадзора, встретился с руководством профессиональной соцсети LinkedIn, которая была заблокирована в РФ в связи с тем, что нарушила правила хранения ...",tech,2
2200,На концерте музыканта Стаса Барецкого и участника реалити-шоу Николая Должанского девушка брызнула им в лица из перцового баллончика.\n\n11:19,incident,2


In [29]:
data[ data['cluster']==3 ].sample(10)

,text,tag,cluster
1499,Народный депутат Надежда Савченко не согласовывает свои действия и поступки с фракцией и партией «Батьківщина». Об этом на брифинге сообщила лидер политического объединения Юлия Тимошенко. Так она...,politics,3
1495,"112 Украина\n\nНадежду Савченко могут исключить из ""Батькивщины"". Об этом в эфире телеканала ""112 Украина"" заявила лидер партии Юлия Тимошенко. Так прокомментировала встречу Савченко с главарями Д...",politics,3
1496,"Юлия Тимошенко, которая является главной украинской партии ""Батьковщина"", заявила, что нардеп Надежда Савченко может быть исключена из партии в ближайшее время.\n\nПо словам Тимошенко, в скором вр...",politics,3
1493,"Лидер украинской партии ""Батькивщина"" Юлия Тимошенко заявила о том, что в ближайшее время будет рассмотрен вопрос об исключении из рядов политического объединения депутата Надежды Савченко.\n\n""Са...",politics,3
1510,"Фракция Батькивщина соберется в первый день парламентской недели, чтобы обсудить действия Надежды Савченко\n\nГлава фракции Батькивщина Юлия Тимошенко заявила, что у депутата Надежды Савченко ""нет...",politics,3
1468,"Лидер Радикальной партии Украины Олег Ляшко обвинил нардепа Верховной рады Надежду Савченко в государственной измене за то, что она встретилась с главами самопровозглашенных ДНР и ЛНР. Об этом Ляш...",politics,3
1503,"\r\n""Батькивщина"" на заседании фракции обсудит вопросы возможного исключения из нее народного депутата Надежды Савченко. Об этом в программе ""Вечерний прайм"" заявила председатель фракции ""Батькивщ...",politics,3
1522,"КИЕВ, 12 дек — РИА Новости. Депутат Верховной рады Украины от партии ""Батькивщина"" Надежда Савченко, ранее осужденная в России по делу об убийстве журналистов в Донбассе и помилованная президентом...",politics,3
1512,"Народный депутат фракции «Батькивщина» Надежда Савченко ответила на обвинения о своей работе на РФ. Об этом она сказала в эфире телеканала NewsOne в понедельник, комментируя просьбу ведущей проком...",politics,3
1463,"Ляшко: встреча Савченко с главами ДНР и ЛНР - это государственная измена\n\nТакое мнение лидер радикальной партии Украины высказал на своей странице в Facebook.\n\n""Встреча депутата Верховной рады...",politics,3


In [30]:
data[ data['cluster']==4 ].sample(10)

,text,tag,cluster
441,"Горбачев: В развале СССР виновато прежнее руководство России\n\n7 декабря 2016 в 21:49\n\nРусская служба Би-би-си\n\nЭкс-президент СССР Михаил Горбачев считает, что в развале Советского Союза вино...",politics,4
1459,Венесуэла закрыла границу с Колумбией\r\n\r\n\r\nПрезидент Венесуэлы Николас Мадуро отдал приказ закрыть границу с Колумбией на 72 часа и использовать их для борьбы с контрабандой валюты.\r\n\r\nК...,politics,4
1678,"Заявление первого президента СССР Михаила Горбачева о том, что в границах бывшего Советского Союза может появиться новое государство, — не более чем популизм. Такое мнение в беседе с «Лентой.ру» в...",politics,4
1407,"Первый президент СССР Михаил Горбачев допустил, что возможно восстановление союза государств, входивших в состав СССР, в прежних границах. Об этом он сообщил в интервью ТАСС.\n\n»[Восстановление в...",politics,4
1460,"Президент Венесуэлы Николас Мадуро распорядился на 72 часа закрыть границу с Колумбией, чтобы избежать контрабанды валюты. Об этом сообщает Associated Press.\n\nПо словам Мадуро, контрабандисты, к...",politics,4
1403,"Горбачев заявил о возможном создании в границах бывшего СССР нового Союза\n\nТелеканал «Россия – Вологда»\n\nЭкс-президент Советского Союза Михаил Горбачев заявил, что в границах бывшего СССР може...",politics,4
1408,"Экс-президент СССР Михаил Горбачев допустил возможность появления в границах бывшего СССР нового Союзного государства.\n\n6308 Горбачёв\n\nСССР\n\nОб этом он заявил в интервью ТАСС , приуроченном ...",politics,4
1461,"В понедельник Николас Мадуро объявил о закрытии границы с Колумбией на 72 часа в связи с тем, что многие жители Венесуэлы спешат обменять деньги до того, как они станут недействительными.\n\nВенес...",politics,4
1673,"Горбачев увидел возможность восстановления Союза в прежних границах\n\nОбъединение, по его словам, может стать полностью добровольным\n\nПервый президент СССР Михаил Горбачев заявил о возможности ...",politics,4
1458,04:03 - 13.12.2016\n\nВенесуэльский лидер Николас Мадуро приказал закрыть границу с Колумбией на 72 часа с целью борьбы с финансовой мафией.\n\nКак заявил глава государства в ходе телевизионного о...,politics,4


In [ ]:
data[ data['cluster']==9 ].sample(10)